In [4]:
# =============================================================================
# STOCK PRICE vs FUNDAMENTAL ANALYSIS
# =============================================================================

import os
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from scipy import stats
from statsmodels.stats.outliers_influence import variance_inflation_factor


# =============================================================================
# CONFIGURATION
# =============================================================================

OUTPUT_DIR = "output/"
os.makedirs(OUTPUT_DIR, exist_ok=True)

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)

file_path = "Intern test 2 - correlation regression - Copy.xls"


# =============================================================================
# STEP 1: LOAD DATA
# =============================================================================

df_stocks = pd.read_excel(file_path, sheet_name=0)
df_fundamentals = pd.read_excel(file_path, sheet_name=1)

print("Stock shape:", df_stocks.shape)
print("Fundamental shape:", df_fundamentals.shape)


# =============================================================================
# STEP 2: CLEAN STOCK PRICE DATA
# =============================================================================

headers = df_stocks.iloc[0].values
metadata_rows = ['Name', 'ISIN Number', 'Exchng Ticker', 'Current Market Value']

stock_data = []

for col_idx in range(1, len(df_stocks.columns)):
    company_name = headers[col_idx]

    isin_row = df_stocks[df_stocks['Symbol'] == 'ISIN Number']
    isin = isin_row.iloc[0, col_idx] if len(isin_row) > 0 else None

    price_data = df_stocks[~df_stocks['Symbol'].isin(metadata_rows)]
    prices = pd.to_numeric(price_data.iloc[:, col_idx], errors='coerce')
    prices_clean = prices.dropna()

    if len(prices_clean) > 0:
        stock_data.append({
            'Company': company_name,
            'ISIN': isin,
            'Avg_Stock_Price': prices_clean.mean()
        })

df_stock_clean = pd.DataFrame(stock_data)


# =============================================================================
# STEP 3: CALCULATE FUNDAMENTAL METRICS
# =============================================================================

def get_field(data, field_name):
    temp = data[data['Field'] == field_name]
    return temp.iloc[0] if not temp.empty else None


metrics_data = []

for company in df_fundamentals['Company name'].unique():

    company_data = df_fundamentals[df_fundamentals['Company name'] == company]
    isin = company_data['ISIN'].iloc[0]

    sales = get_field(company_data, 'SALES')
    ebitda = get_field(company_data, 'EBITDA')
    pat = get_field(company_data, 'PAT')

    if sales is None or ebitda is None or pat is None:
        continue

    try:
        sales_growth = ((sales[2024] - sales[2023]) / abs(sales[2023]) * 100) if sales[2023] != 0 else np.nan
        ebitda_growth = ((ebitda[2024] - ebitda[2023]) / abs(ebitda[2023]) * 100) if ebitda[2023] != 0 else np.nan
        pat_growth = ((pat[2024] - pat[2023]) / abs(pat[2023]) * 100) if pat[2023] != 0 else np.nan

        ebitda_margin_2024 = (ebitda[2024] / sales[2024] * 100) if sales[2024] != 0 else 0
        ebitda_margin_2023 = (ebitda[2023] / sales[2023] * 100) if sales[2023] != 0 else 0
        ebitda_margin_change = ebitda_margin_2024 - ebitda_margin_2023

        pat_margin_2024 = (pat[2024] / sales[2024] * 100) if sales[2024] != 0 else 0
        pat_margin_2023 = (pat[2023] / sales[2023] * 100) if sales[2023] != 0 else 0
        pat_margin_change = pat_margin_2024 - pat_margin_2023

        metrics_data.append({
            'Company': company,
            'ISIN': isin,
            'Sales_Growth': sales_growth,
            'EBITDA_Margin_Change': ebitda_margin_change,
            'EBITDA_Growth': ebitda_growth,
            'PAT_Growth': pat_growth,
            'PAT_Margin_Change': pat_margin_change
        })

    except:
        continue

df_metrics = pd.DataFrame(metrics_data)


# =============================================================================
# STEP 4: MERGE DATA
# =============================================================================

df_final = pd.merge(df_stock_clean, df_metrics, on='ISIN')
df_final = df_final.drop(columns=['Company_x'])
df_final = df_final.rename(columns={'Company_y': 'Company'})

df_final = df_final.dropna()


fundamental_vars = [
    'Sales_Growth',
    'EBITDA_Margin_Change',
    'EBITDA_Growth',
    'PAT_Growth',
    'PAT_Margin_Change'
]

print("Merged dataset:", df_final.shape)


# =============================================================================
# STEP 5: PRE-TESTS
# =============================================================================

print("\nDescriptive Statistics")
print(df_final[['Avg_Stock_Price'] + fundamental_vars].describe())

print("\nMissing Values")
print(df_final.isnull().sum())

print("\nOutlier Detection (|Z| > 2)")
for col in ['Avg_Stock_Price'] + fundamental_vars:
    z = np.abs(stats.zscore(df_final[col]))
    print(col, "Outliers:", (z > 2).sum())

print("\nNormality Test (Shapiro)")
for col in ['Avg_Stock_Price'] + fundamental_vars:
    if len(df_final[col]) >= 3:
        stat, p = stats.shapiro(df_final[col])
        print(col, "p-value:", round(p, 4))

print("\nMulticollinearity (VIF)")
X_vif = df_final[fundamental_vars].values
for i, var in enumerate(fundamental_vars):
    try:
        vif = variance_inflation_factor(X_vif, i)
        print(var, "VIF:", round(vif, 2))
    except:
        print(var, "VIF: N/A")


# =============================================================================
# STEP 6: CORRELATION
# =============================================================================

corr_matrix = df_final[['Avg_Stock_Price'] + fundamental_vars].corr()
print("\nCorrelation Matrix")
print(corr_matrix)

plt.figure(figsize=(10,8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0)
plt.title("Correlation Matrix")
plt.tight_layout()
plt.savefig(OUTPUT_DIR + "correlation_heatmap.png")
plt.close()


# =============================================================================
# STEP 7: SIMPLE LINEAR REGRESSIONS
# =============================================================================

y = df_final['Avg_Stock_Price'].values
n = len(y)

results = []

for var in fundamental_vars:

    X = df_final[[var]].values
    model = LinearRegression()
    model.fit(X, y)

    y_pred = model.predict(X)
    r2 = model.score(X, y)

    ss_total = np.sum((y - np.mean(y))**2)
    ss_res = np.sum((y - y_pred)**2)
    ss_reg = ss_total - ss_res

    df_reg = 1
    df_res = n - 2

    if df_res > 0 and ss_res > 0:
        ms_reg = ss_reg / df_reg
        ms_res = ss_res / df_res
        f_stat = ms_reg / ms_res
        p_val = 1 - stats.f.cdf(f_stat, df_reg, df_res)
    else:
        f_stat = np.nan
        p_val = np.nan

    results.append({
        'Variable': var,
        'Coefficient': model.coef_[0],
        'Intercept': model.intercept_,
        'R_squared': r2,
        'F_stat': f_stat,
        'P_value': p_val
    })

df_results = pd.DataFrame(results)
print("\nRegression Results")
print(df_results)

top3 = df_results.sort_values("P_value").head(3)
print("\nTop 3 Significant Variables")
print(top3)


# =============================================================================
# STEP 8: MULTIPLE REGRESSION
# =============================================================================

X_multi = df_final[fundamental_vars].values
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_multi)

model_multi = LinearRegression()
model_multi.fit(X_scaled, y)

print("\nMultiple Regression R²:", model_multi.score(X_scaled, y))
# =============================================================================
# PLOT: STOCK PRICE vs EACH FUNDAMENTAL VARIABLE
# =============================================================================


print("\nGenerating Scatter Plots...")

y = df_final['Avg_Stock_Price'].values

for var in fundamental_vars:

    X = df_final[[var]].values
    model = LinearRegression()
    model.fit(X, y)

    y_pred = model.predict(X)
    r2 = model.score(X, y)

    plt.figure(figsize=(8,6))

    # Scatter
    plt.scatter(df_final[var], y, s=120, edgecolor='black')

    # Regression line
    x_line = np.linspace(df_final[var].min(), df_final[var].max(), 100)
    y_line = model.predict(x_line.reshape(-1,1))
    plt.plot(x_line, y_line, linestyle='--')

    # Annotate company names
    for i, txt in enumerate(df_final['Company']):
        short_name = str(txt).split()[0]
        plt.annotate(short_name,
                     (df_final[var].iloc[i], y[i]),
                     textcoords="offset points",
                     xytext=(5,5),
                     fontsize=8)

    plt.xlabel(var.replace("_"," "))
    plt.ylabel("Average Stock Price")
    plt.title(f"Stock Price vs {var.replace('_',' ')}\nR² = {r2:.3f}")
    plt.tight_layout()

    plt.savefig(OUTPUT_DIR + f"scatter_{var}.png")
    plt.close()

print("Scatter plots saved in /output folder.")



# =============================================================================
# STEP 9: R-SQUARED BAR CHART
# =============================================================================

plt.figure()
plt.bar(df_results['Variable'], df_results['R_squared'])
plt.xticks(rotation=45)
plt.title("R-squared Comparison")
plt.tight_layout()
plt.savefig(OUTPUT_DIR + "r_squared_comparison.png")
plt.close()


# =============================================================================
# STEP 10: EXPORT TO EXCEL
# =============================================================================

with pd.ExcelWriter(OUTPUT_DIR + "analysis_results.xlsx", engine='openpyxl') as writer:
    df_final.to_excel(writer, sheet_name="Merged_Data", index=False)
    corr_matrix.to_excel(writer, sheet_name="Correlation")
    df_results.to_excel(writer, sheet_name="Regression_Results", index=False)
    top3.to_excel(writer, sheet_name="Top3", index=False)

print("\nAll outputs saved in /output folder.")


Stock shape: (4940, 6)
Fundamental shape: (15, 33)
Merged dataset: (5, 8)

Descriptive Statistics
       Avg_Stock_Price  Sales_Growth  EBITDA_Margin_Change  EBITDA_Growth  \
count         5.000000      5.000000              5.000000       5.000000   
mean        706.563985      3.670543              0.871922      12.229051   
std         530.170315      3.044392              1.772973      15.854696   
min         122.335466     -0.748549             -0.469555       3.811691   
25%         515.912078      1.909396             -0.206578       4.251145   
50%         626.556417      4.703373             -0.204733       5.177616   
75%         701.091600      5.990627              1.538893       7.423789   
max        1566.924364      6.497866              3.701582      40.481013   

       PAT_Growth  PAT_Margin_Change  
count    5.000000           5.000000  
mean    24.925082           1.422181  
std     31.345021           1.480067  
min      5.761523          -0.041193  
25%      8.87